<a href="https://colab.research.google.com/github/RosalieWessels/InspiritAI---Maze-Solver/blob/main/Rosa_Maze_Solver_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌀 **Let's solve mazes!**
Our goal in this notebook is to use search algorithms we've learned in class to find the shortest path through a maze. We'll start by implementing DFS, then move on to BFS. Similar to minesweeper, we're going to be working with a visual representation of our maze as well as a separate representation in code.

In [ ]:
#@title Run this cell to download some of the mazes!
!gdown -q 1_VXIk8bEK7qO6BsqXjDU6L7WBsNdTbMH
!gdown -q 1nLW-8qBywe7n8LK1H-XepJYrFhh0xvVT
!gdown -q 1hBAqGd-yPQzui5EFswwrJy1yjNakKH0q
!gdown -q 1q7i6jSkkNJ8cpDplgKJFe27vSwXdJljC

In [ ]:
#@title Run this cell to set up the graphics functions!
from IPython.display import display, HTML
from PIL import Image
import time
import math
import re
import random
import copy


# Module for drawing classic Turtle figures on Google Colab notebooks.
# It uses html capabilites of IPython library to draw svg shapes inline.

# DEFAULT_BACKGROUND_COLOR = 'white'
DEFAULT_SVG_LINES_STRING = ""
# COLORS = ("ForestGreen", "LimeGreen")
SVG_TEMPLATE = """
      <svg width={window_width} height={window_height}>
        {image}
        {updates}
      </svg>
    """

drawing_window = None
svg_image_string = ''

def make_maze(maze_dims = None, maze_size = "small"):
  global window_size
  global svg_image_string
  global drawing_window

  st = """<defs>
        <pattern id="image" patternUnits="userSpaceOnUse" width={width} height={height}>"""
  en = """</pattern>
      </defs>
      <rect id='top' x="0" y="0" width={width} height={height} fill="url(#image)"/>"""

  if maze_size == "small":
    window_size = (80, 80)
    svg_image_string = st.format(width=window_size[0], height=window_size[1]) + '<image x="0" y="0" xlink:href="https://drive.google.com/uc?export=view&id=1Y5fRXpRHYiwiUtSVj8eQwE86x1iQhdVv"></image>' + en.format(width=window_size[0], height=window_size[1])
    im = Image.open("small_maze.png")
    start_loc = (5, 6)
    goal_loc = (1, 1)
  elif maze_size == "medium":
    window_size = (190, 190)
    svg_image_string = st.format(width=window_size[0], height=window_size[1]) + '<image x="0" y="0" xlink:href="https://drive.google.com/uc?export=view&id=1fhnydLvhs3wvO0fryawtHouYi3iH6vBu"></image>' + en.format(width=window_size[0], height=window_size[1])
    im = Image.open("medium_maze.png")
    start_loc = (17, 17)
    goal_loc = (1, 1)
  elif maze_size == "large":
    window_size = (490, 490)
    svg_image_string = st.format(width=window_size[0], height=window_size[1]) + '<image x="0" y="0" xlink:href="https://drive.google.com/uc?export=view&id=1bvu1JBovT8MYN8JccP2Wz61LQCKI6VoM"></image>' + en.format(width=window_size[0], height=window_size[1])
    im = Image.open("large_maze.png")
    start_loc = (47, 47)
    goal_loc = (1, 1)
  elif maze_size == "no_connect":
    window_size = (80, 80)
    start_loc = (6, 1)
    goal_loc = (1, 6)

    svg_image_string = """<rect x="{x}" y="{y}", width="80" height="80" style="fill:{fill}" />""".format(x=0, y=0, fill="Black")
    svg_image_string += """<rect x="{x}" y="{y}", width="60" height="60" style="fill:{fill}" />""".format(x=10, y=10, fill="White")
    svg_image_string += """<rect x="{x}" y="{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*start_loc[1], y=10*start_loc[0], fill="Lime")
    for i in range(1, 7):
      svg_image_string += """<rect x="{x}" y="{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*i, y=10*i, fill="Black")
    
  elif maze_size == "rand":
    window_size = (80, 80)
    svg_image_string = """<rect x="{x}" y="{y}", width="80" height="80" style="fill:{fill}" />""".format(x=0, y=0, fill="Black")
    list_maze = [['|', '|', '|', '|', '|', '|', '|', '|']]
    for i in range(1, 7):
      row = ['|']
      for j in range(1, 7):
        ch = random.choice(['|', 'O'])
        row.append(ch)
        if ch == "O":
          svg_image_string += """<rect x="{x}" y="{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*j, y=10*i, fill="White")
      row.append('|')
    list_maze.append(['|', '|', '|', '|', '|', '|', '|', '|'])
    start_loc = (6, 1)
    goal_loc = (1, 6)
    svg_image_string += """<rect x="{x}" y="{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*start_loc[1], y=10*start_loc[0], fill="Lime")

  elif maze_size == "blank":
    if maze_dims != None:
      window_size = (10*maze_dims[1], 10*maze_dims[0])
      svg_image_string = """<rect x="0" y="0", width="{w}" height="{h}" style="fill:{fill}" />""".format(w=10*maze_dims[1], h=10*maze_dims[0], fill="Black")
    else:
      print("Maze dims not valid.")
      return None

  else:
    print("Maze size not valid. Please use 'small', 'medium', 'large', 'no_connect', or 'rand'.")
    return None

  if maze_size=="blank":
    updates = ''
  else:
    updates = """<rect x="{x}" y="{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*goal_loc[1], y=10*goal_loc[0], fill="Aqua")
  drawing_window = display(HTML(_generateSvgDrawing(up=updates)), display_id=True)

  list_maze = []
  wsize = (int(window_size[0]/10), int(window_size[1]/10))
  if maze_size not in ["no_connect", "rand", "blank"]:
    for i in range(wsize[0]):
      row = []
      for j in range(wsize[1]):
        val = im.getpixel((j, i))[0]
        if val == 255 or (i,j)==start_loc or (i,j)==goal_loc:
          val = "O"
        else:
          val = "|"
        row.append(val)
      list_maze.append(row)
  elif maze_size=="no_connect":
    list_maze = [['|', '|', '|', '|', '|', '|', '|', '|'],
                 ['|', '|', 'O', 'O', 'O', 'O', 'O', '|'],
                 ['|', 'O', '|', 'O', 'O', 'O', 'O', '|'],
                 ['|', 'O', 'O', '|', 'O', 'O', 'O', '|'],
                 ['|', 'O', 'O', 'O', '|', 'O', 'O', '|'],
                 ['|', 'O', 'O', 'O', 'O', '|', 'O', '|'],
                 ['|', 'O', 'O', 'O', 'O', 'O', '|', '|'],
                 ['|', '|', '|', '|', '|', '|', '|', '|'],
                 ]
  elif maze_size == "blank":
    list_maze = []
    for i in range(maze_dims[0]):
      row = []
      for j in range(maze_dims[1]):
        row.append('|')
      list_maze.append(row)
    return list_maze
  return list_maze, start_loc, goal_loc


# helper function for generating the whole svg string
def _generateSvgDrawing(up=''):
  # print(SVG_TEMPLATE.format(window_width=window_size[0], window_height=window_size[1],lines=svg_lines_string))
  return SVG_TEMPLATE.format(window_width=window_size[0], window_height=window_size[1],image=svg_image_string,updates=up)


# helper functions for updating the screen using the latest positions/angles/lines etc.
def _updateDrawing(up='', speed = 'normal'):
    if drawing_window == None:
        raise AttributeError("Display has not been initialized yet.")
    time.sleep(0.2)
    drawing_window.update(HTML(_generateSvgDrawing(up)))

def update_graphics(goal, seen=set(), path=[], speed = 'normal'):
  updates = """<rect x="{x}" y="{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*goal[1], y=10*goal[0], fill="Aqua")
  for (r, c) in path[1:-1]:
    updates += """<rect x="{x}" y = "{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*c, y=10*r, fill="Red")
  for (r, c) in seen:
    updates += """<rect x="{x}" y = "{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*c, y=10*r, fill="Thistle")
  _updateDrawing(up=updates, speed=speed)

def remove_wall(maze, loc, n):
  updates = ''
  wall_rem = (int((loc[0]-n[0])/2) + n[0], int((loc[1]-n[1])/2) + n[1])
  maze[loc[0]][loc[1]] = 'O'
  maze[n[0]][n[1]] = 'O'
  maze[wall_rem[0]][wall_rem[1]] = 'O'
  for i in range(len(maze)):
    for j in range(len(maze[0])):
      if maze[i][j]=='O':
        updates +="""<rect x="{x}" y="{y}", width="10" height="10" style="fill:{fill}" />""".format(x=10*j, y=10*i, fill="White")
  _updateDrawing(up=updates)
  return maze

def maze_gen_neighbors(row, col, num_rows, num_cols, seen):
  init_n = [(row+2, col), (row-2, col), (row, col+2), (row, col-2)]
  true_n = []
  for n in init_n:
    if n[0]>=1 and n[0]<num_rows-1 and n[1]>=1 and n[1]<num_cols-1 and n not in seen:
      true_n.append(n)
  return true_n

# 📊 **Graph Representation**
Let's use the `make_maze` function to take a look at our maze. This function does two important things things - it creates and shows us the visual representation of the maze, and it also returns three things:
1. the python representation of the maze, formatted as a 2D list
2. the start location, as a tuple of `(row, column)`
3. the end or goal location, as a tuple of `(row, column)`

There are four mazes available to us - `"small"`, `"medium"`, `"large"`, `"no_connect"`, and `'rand'`. Note that the `'rand'` maze sets locations to be open or a wall randomly, so most of the time there won't be a path from the start to the goal.


The cell below will create and show us the small maze. After looking at the small maze, use the cell below to check out the others.



In [ ]:
# change the maze_size variable to see different types of mazes!
maze, start, goal = make_maze(maze_size = "small")

Visually, it's fairly easy to understand what's going on with the maze. All the **black** cells (locations) are walls, **white** cells are open, the **green** cell is the starting point, and the cyan **blue** cell is the goal location.

Let's take a look at the code representation, which is a bit different! Run the cell below to peek at our maze as a 2D python list of strings.

In [ ]:
print("Starting Location:", start)
print("Goal Location:", goal)
for row in maze:
  print(row)

Starting Location: (5, 6)
Goal Location: (1, 1)
['|', '|', '|', '|', '|', '|', '|', '|']
['|', 'O', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', 'O', '|', '|', 'O', '|', '|']
['|', 'O', 'O', '|', 'O', '|', '|', '|']
['|', '|', 'O', 'O', 'O', '|', 'O', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', '|', '|', '|', '|', '|', '|']


In the python representation, our maze is a 2D list of `'|'`, representing walls in the maze, and `'O'`, representing open spaces in the maze (that is a capital O, not a zero). We will be using the python representation for our code, and the graphics representation to visualize where our algorithm is searching and the final path it finds.

## 💡 **Check yourself!**

<img src="https://drive.google.com/uc?export=view&id=1Hh-nbjtUNxd8IqRNgQOhxP-rCy8zn7GG" 
     width="200" 
     height="auto" />

#### **What is the corresponding python representation of this map?**

In [ ]:
# Fill in the open spaces in the maze, start, and goal in code here!
above_maze = [['|', '|', '|', '|', '|', '|', '|', '|'],
              ['|', '|', '|', 'O', 'O', '|', 'O', '|'],
              ['|', '|', '|', 'O', 'O', 'O', 'O', '|'],
              ['|', 'O', 'O', '|', 'O', 'O', 'O', '|'],
              ['|', '|', '|', 'O', 'O', 'O', 'O', '|'],
              ['|', 'O', 'O', 'O', '|', 'O', '|', '|'],
              ['|', 'O', 'O', '|', '|', 'O', 'O', '|'],
              ['|', '|', '|', '|', '|', '|', '|', '|'],
              ]
above_start = (6, 1)
above_goal = (1, 6)

In [ ]:
#@title Run this cell to check your solution!
above_maze_sol = [['|', '|', '|', '|', '|', '|', '|', '|'],
              ['|', '|', '|', 'O', 'O', '|', 'O', '|'],
              ['|', '|', '|', 'O', 'O', 'O', 'O', '|'],
              ['|', 'O', 'O', '|', 'O', 'O', 'O', '|'],
              ['|', '|', '|', 'O', 'O', 'O', 'O', '|'],
              ['|', 'O', 'O', 'O', '|', 'O', '|', '|'],
              ['|', 'O', 'O', '|', '|', 'O', 'O', '|'],
              ['|', '|', '|', '|', '|', '|', '|', '|'],
              ]
above_start_sol = (6, 1)
above_goal_sol = (1, 6)
if above_maze == above_maze_sol:
  print("Your maze is correct!")
else:
  print("Your maze is NOT correct.")

if above_start == above_start_sol:
  print("Your starting location is correct!")
else:
  print("Your starting location is NOT correct.")

if above_goal == above_goal_sol:
  print("Your ending location is correct!")
else:
  print("Your ending location is NOT correct.")

Your maze is correct!
Your starting location is correct!
Your ending location is correct!


In [ ]:
#@title **Is there a valid path between the start and the goal?**
is_path1 = "Yes" #@param ["", "Yes", "No"]

if is_path1 == "Yes":
  print("That's correct!")
else:
  print("Your answer is incorrect")

That's correct!


If there is a valid path, fill in the list below with what a path could be (as a list of `(row, col)` tuples including the start and goal location). Otherwise, leave the list empty. There might be more than one right solution!

In [ ]:
# fill in the list below!
path1 = [(6,1), (6, 3), (5, 3), (5, 4), (4, 4), (4, 5), (4, 6), (3, 6), (2, 6), (1, 6)]

## 💡 **Check yourself!**

<img src="https://drive.google.com/uc?export=view&id=1aSTEgotN72dqgriIFR2Mdw9n-6l9Q5Ik" 
     width="200" 
     height="auto" />

### What is the corresponding python representation of this map?

In [ ]:
# Fill in the maze, start, and goal in code here!
above_maze = [['|', '|', '|', '|', '|', '|', '|', '|'],
              ['|', 'O', 'O', '|', '|', 'O', 'O', '|'],
              ['|', 'O', '|', '|', 'O', 'O', '|', '|'],
              ['|', 'O', 'O', 'O', 'O', 'O', 'O', '|'],
              ['|', '|', '|', '|', '|', 'O', '|', '|'],
              ['|', '|', 'O', 'O', '|', '|', 'O', '|'],
              ['|', 'O', 'O', 'O', '|', 'O', '|', '|'],
              ['|', '|', '|', '|', '|', '|', '|', '|'],
              ]
above_start = (6, 1)
above_goal = (1, 6)

In [ ]:
#@title **Is there a valid path between the start and the goal?**
is_path2 = "No" #@param ["", "Yes", "No"]

if is_path2 == "No":
  print("That's correct!")
else:
  print("Your answer is incorrect")

That's correct!


If there is a valid path, fill in the list below with what a path could be (as a list of `(row, col)` tuples including the start and goal location). Otherwise, leave the list empty.

In [ ]:
# fill in the list below!
path2 = []

# 👭 **Neighbors**
For the purposes of this problem, we will **NOT** consider diagonal adjacent tiles as neighbors. This means that when you're looking at the neighbors of a map location, it should have at **most** four neighbors. Remember, "neighboring" locations that are walls (`'|'`) and not open (`'O'`) **are not considered valid neighbors either**. Any location that has a wall **has no valid neighbors.**

In short, the neighbors list for your location should only have the cell above, below, and directly to the right and left **if** those cells are open. 

## 💡 **Check yourself!**

Below is the python representation of the `'small'` maze.

```python
['|', '|', '|', '|', '|', '|', '|', '|']
['|', 'O', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', 'O', '|', '|', 'O', '|', '|']
['|', 'O', 'O', '|', 'O', '|', '|', '|']
['|', '|', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', '|', '|', '|', '|', '|', '|']
```

What are the neighbors of maze location `(1, 1)`?


In [ ]:
neighbors_11 = [(1, 2)] # fill this in!

```python
['|', '|', '|', '|', '|', '|', '|', '|']
['|', 'O', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', 'O', '|', '|', 'O', '|', '|']
['|', 'O', 'O', '|', 'O', '|', '|', '|']
['|', '|', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', '|', '|', '|', '|', '|', '|']
```

What are the neighbors of maze location `(2, 5)`?

In [ ]:
neighbors_25 = [(2, 4), (2, 6), (3, 5)] # fill this in!

```python
['|', '|', '|', '|', '|', '|', '|', '|']
['|', 'O', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', 'O', '|', '|', 'O', '|', '|']
['|', 'O', 'O', '|', 'O', '|', '|', '|']
['|', '|', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', '|', '|', '|', '|', '|', '|']
```

What are the neighbors of maze location `(3, 1)`?

In [ ]:
neighbors_31 = [] # fill this in!

```python
['|', '|', '|', '|', '|', '|', '|', '|']
['|', 'O', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', 'O', '|', '|', 'O', '|', '|']
['|', 'O', 'O', '|', 'O', '|', '|', '|']
['|', '|', 'O', 'O', 'O', '|', '|', '|']
['|', '|', 'O', '|', 'O', 'O', 'O', '|']
['|', '|', '|', '|', '|', '|', '|', '|']
```

What are the neighbors of maze location `(2, 2)`?

In [ ]:
neighbors_22 = [(1, 2), (3, 2)] # fill this in!

In [ ]:
#@title Run this to check your answers!
if neighbors_11==[(1,2)]:
  print('Your neighbors for (1, 1) are correct!')
else:
  print('Your neighbors for (1, 1) are NOT correct!')

if set(neighbors_25)=={(2, 4), (2, 6), (3, 5)}:
  print('Your neighbors for (2, 5) are correct!')
else:
  print('Your neighbors for (2, 5) are NOT correct!')

if neighbors_31==[]:
  print('Your neighbors for (3, 1) are correct!')
else:
  print('Your neighbors for (3, 1) are NOT correct!')

if set(neighbors_22)=={(1,2), (3, 2)}:
  print('Your neighbors for (2, 2) are correct!')
else:
  print('Your neighbors for (2, 2) are NOT correct!')


Your neighbors for (1, 1) are correct!
Your neighbors for (2, 5) are correct!
Your neighbors for (3, 1) are correct!
Your neighbors for (2, 2) are correct!


## 🏡 **Neighbors Implementation**
Now it's time to implement the neighbors function. In the cell below, fill in the `neighbors` function definition. This function takes in a `maze`, `row`, and `col`. It should return the neighboring locations of `(row, col)` in the maze. 

**A few things to note**:
* Diagonal adjacent cells are NOT neighbors
* Cells that are walls are NOT neighbors

In [ ]:
def neighbors(maze, row, col):
  '''
  This function takes in a 2D maze and (row, col) maze location and should return a list of
  all of the valid neighboring locations in the board.
  '''
  if maze[row][col] == "|":
    return []

  possible_neighbors = [(row, col - 1), (row, col + 1), (row - 1, col), (row + 1, col)]
  neighbors = []
  for n in possible_neighbors:
    n_row, n_col = n
    if maze[n_row][n_col] == "O":
      neighbors.append((n_row, n_col))

  return neighbors

In [ ]:
# use this cell to check your neighbors function!
above_maze = [['|', '|', '|', '|', '|', '|', '|', '|'],
              ['|', 'O', 'O', '|', '|', 'O', 'O', '|'],
              ['|', 'O', '|', '|', 'O', 'O', '|', '|'],
              ['|', 'O', 'O', 'O', 'O', 'O', 'O', '|'],
              ['|', '|', '|', '|', '|', 'O', '|', '|'],
              ['|', '|', 'O', 'O', '|', '|', 'O', '|'],
              ['|', 'O', 'O', 'O', '|', 'O', '|', '|'],
              ['|', '|', '|', '|', '|', '|', '|', '|'],
              ]
above_start = (6, 1)
above_goal = (1, 6)
print(neighbors(above_maze, 0,0))

[]


# 🔎 **Search: DFS**
Now that we have a good understanding of how our maze is represented, let's code our search algorithms! We're going to start by coding DFS, or Depth First Search. Fill in the function below to implement this algorithm.

**NOTE:** your function does not have to find the *shortest* path, just any *valid* path from `start` to `goal` or from `goal` to `start` if that's easier.


Remember that since you need to return a *path* you will need to use one of the methods we talked about in class - either a `came_from` dictionary or adding partial paths to your queue.

In [ ]:
def maze_DFS(maze, start, goal):
  '''
  Solve the given maze by using DFS. 
  Parameters:
    maze: a maze as a 2D list as described above
    start: the starting location for search, as a tuple in the form (row, col)
    goal: the ending location for search, as a tuple in the form (row, col)
  Returns:
    list of tuples representing a valid path through the maze
  '''
  # initialize queue, seen set, current node/path and came_from if you're using it
  queue = [start]
  current = None
  seen = set()
  path_dict = {}
  # create loop
  while len(queue) > 0:
    # take item off queue
    current = queue[0]
    queue.remove(current)
    seen.add(current)
    # look at neighbors of item
    c_row, c_col = current
    ns = neighbors(maze, c_row, c_col)
    for n in ns:
      # if neighbor is goal, return path or break
      if n == goal:
        path_dict[goal] = current
        return find_path(maze, start, goal, path_dict)
      # if neighbor has already been seen, do nothing
      if n not in seen:
        queue.insert(0, n)
        path_dict[n] = current
        seen.add(n)
        
  
def find_path(maze, start, goal, path_dict):
  path = [goal]
  index = 0
  while start not in path:
    next = path_dict.get(path[index])
    path.append(next)
    index += 1

  return path

Once you've finished your DFS function, use the cell below to test it out. Make sure your function works on all mazes, not just the small one!

In [ ]:
# RUN THIS to test out your function!
maze, start, goal = make_maze(maze_size = "large") # change the map size to see different ones!
path = maze_DFS(maze, start, goal)
print(path)
if path != None:
  update_graphics(goal, path=path)
else:
  print("No path found!")

[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (2, 32), (3, 32), (4, 32), (5, 32), (6, 32), (7, 32), (7, 33), (7, 34), (7, 35), (7, 36), (7, 37), (7, 38), (7, 39), (7, 40), (6, 40), (5, 40), (5, 39), (5, 38), (5, 37), (5, 36), (4, 36), (3, 36), (2, 36), (1, 36), (1, 37), (1, 38), (2, 38), (3, 38), (3, 39), (3, 40), (3, 41), (3, 42), (3, 43), (2, 43), (1, 43), (1, 44), (1, 45), (2, 45), (3, 45), (4, 45), (5, 45), (6, 45), (7, 45), (8, 45), (9, 45), (9, 46), (9, 47), (10, 47), (11, 47), (12, 47), (13, 47), (14, 47), (15, 47), (16, 47), (17, 47), (18, 47), (18, 46), (18, 45), (19, 45), (20, 45), (20, 44), (20, 43), (20, 42), (20, 41), (20, 40), (20, 39), (20, 38), (19, 38), (18, 38), (17, 38), (16, 38), (16, 39), (16, 40), (16, 41), (16, 42), (16, 43), (15, 43

# 🔎 **Search: BFS**
Now we're going to switch to using BFS, or Breadth First Search. Fill in the function below to implement this algorithm.

**NOTE:** your function does not have to find the *shortest* path, just any *valid* path from `start` to `goal` or from `goal` to `start` if that's easier.

Remember that since you need to return a *path* you will need to use one of the methods we talked about in class - either a `came_from` dictionary or adding partial paths to your queue.

In [ ]:
def maze_BFS(maze, start, goal):
  '''
  Solve the given maze by using BFS. 
  Parameters:
    maze: a maze as a 2D list as described above
    start: the starting location for search, as a tuple in the form (row, col)
    goal: the ending location for search, as a tuple in the form (row, col)
  Returns:
    list of tuples representing a valid path through the maze
  '''
  # initialize queue, seen set, current node/path and came_from if you're using it
  queue = [start]
  current = None
  seen = set()
  path_dict = {}
  # create loop
  while len(queue) > 0:
    # take item off queue
    current = queue[0]
    queue.remove(current)
    seen.add(current)
    # look at neighbors of item
    c_row, c_col = current
    ns = neighbors(maze, c_row, c_col)
    for n in ns:
      # if neighbor is goal, return path or break
      if n == goal:
        path_dict[goal] = current
        return find_path(maze, start, goal, path_dict)
      if n not in seen:
        queue.append(n)
        path_dict[n] = current
        seen.add(n)

Once you've finished your BFS function, use the cell below to test it out. Make sure your function works on all mazes, not just the small one!

In [ ]:
# RUN THIS to test out your function!
maze, start, goal = make_maze(maze_size = "large") # change the map size to see different ones!
path = maze_BFS(maze, start, goal)
if path != None:
  update_graphics(goal, path=path)
else:
  print("No path found!")

# 🧰 **Extension: Maze Generation**

It's great to be able to solve mazes, but we only have a few mazes to work with! What if we want to test our search algorithm on more?

We currently have two options:
1. Make new mazes by hand
2. Use the `'rand'` `maze_size`, which randomly decides if cells in the maze should be a wall or open

Making new mazes by hand guarantees that you are creating mazes that work, but it is a long and tedious process.

Using `'rand'` doesn't always guarantee that you have a solvable maze!

Instead of using one of these methods, we can actually use **DFS** to generate new, unique, maze-like structures!

## 🚁 **Helper Functions**
There will be three helper functions you'll use as you're implementing this function (they'll be referenced in the pseudocode below). These functions are:

* `make_maze`: we've used this function before, but there's one optional parameter we haven't used: `maze_dims`. You can pass in the number of rows and cols you want the maze to be as a tuple for that parameter. It creates a maze with only walls in it, where the size is `(num_rows*2, num_cols*2)`. When we use `make_maze` to make a blank maze, it only returns a maze, not a start and goal location. **Use:**
```python
maze = make_maze(maze_dims = (num_rows, num_cols), maze_size = "blank")
```
* `remove_wall`: this function takes in the python representation of the maze and the two neighboring locations (as tuples of `(row, col)`) on the board and removes the wall between them. **Use:**
```python
updated_maze = remove_wall(maze, loc, neighbor)
```
* `maze_gen_neighbors`: this function takes five parameters: `row`, `col` (representing the location you want the neighbors of), `num_rows`, `num_cols` (the dimensions of the board), and the seen set. **Use:**
```python
neighbors = maze_gen_neighbors(row, col, num_rows, num_cols, seen)
```

## 🔤 **Maze Extension Pseudocode**
Our code will look pretty similar to DFS as we've seen and used it before, but there will be a few key differences. Check out the pseudocode below to help you understand what's going on in this function.

```python
def generate_maze(num_rows, num_cols, start = None):
  create blank maze using init_maze
  make queue, seen set, current
  choose an initial cell, put in seen, add to queue
  while the queue is not empty:
    take the first location off the queue
    if current loc has any neighbours not in seen:
      put the current loc back on the queue
      randomly choose one of those neighbors 
      remove the wall between the current and neighbor
      put that neighbor in seen, add it to the queue
```

Time to implement! Fill in the function definition below.

**Hint:** to choose a random item from a list, you can use `random.choice`. Check out the [usage here](https://www.w3schools.com/python/ref_random_choice.asp).

In [ ]:
import random
def generate_maze(num_rows, num_cols, start=None):
  '''
  This function will return a maze that is created using DFS.
  Use the starting location `start` if one is passed in
  '''
  maze = make_maze(maze_dims = (num_rows, num_cols), maze_size = "blank")
  queue = []
  if start != None:
    initial = start
  else:
    initial = (1, 1)
  current = None
  seen = set()
  queue.append(initial)

  seen.add(current)
  while len(queue) > 0:
    current = queue[0]
    queue.remove(current)
    row, col = current
    new_n = maze_gen_neighbors(row, col, num_rows, num_cols, seen)
    if new_n != []:
      queue.append(current)
      remove_n = random.choice(new_n)
      maze = remove_wall(maze, current, remove_n)
      seen.add(remove_n)
      queue.append(remove_n)


  return maze

Once you've finished the function, go ahead and run the cell to test it out below! 

In [ ]:
maze = generate_maze(21, 31, start=(1,1))

If you want to solve a map you've generated using BFS or DFS like you were earlier, make sure you define a start and goal location! You can choose these values.

In [ ]:
# Solve your maze using this cell here!
# fill in values for your start and goal!
start = (1,1)
goal = (1,25)

# uncomment below to run DFS!
path = maze_BFS(maze, start, goal) # you can change this to BFS
if path != None:
  update_graphics(goal, path=path)
else:
  print("No path found!")

# 🎉 **That's it! You've finished this notebook :)**